# Отчёт 07.02.25

## Задача проекта

**User Story:**

Я, как product owner хочу иметь возможность расчитывать производительность из параметров железа     
Чтобы снизить затраты производства и повысить предсказуемость характеристик продукта

**Критерии приемки:**

* Возможность восстановить параметры железа по производительности;
* Модель должна выводить сравнительные графики: сырой даты и предсказанной;
* Вывод метрик модели (R², средние квадраты и тд.);
* Модель экстраполирует данные на интервал 4 - 100 узлов;
* Выявлено влияние отдельных конфигурационных параметров;
* Выявлено совокупное влияние разных конфигурационных параметров;
* Возможность выгружать замеренные данные формата .txt/.md в .xlsx таблицу;


На текущий момент имеется модель, реализованная с помощью языка Python (3.12.8) с использованием библиотек `pymc`, `arviz`, `numpy`,`pandas`, `itertools` и пр. в среде Jupytrer Notebook. В своей основе модель использует метод байесовского моделирования линейной регрессии, формульно: $${predict} = \boldsymbol{weights} \cdot \boldsymbol{features}.$$

По метрике $R^2$ модель показывает результат .86 и на дистанции ведёт себя согласно предположению о зависимости целевого значения от признаков.

## Данные

В качестве целевого значения рассматривается производительность системы, целевое значение зависит от признаков: 

### Numerical

* `nodes` - количество узлов: $4,\,5,\,\cdots,100$;
* `size` - размер объекта: $4,\,5,\;\cdots\,128e3$ (KB); 
* `SSD` - размер SSD: $2,\;4$;

### Categorial

* `type` - тип действия: запись / чтение; 
* `protection` - защита: REP 2, REP 3, EC2.1;
* `Hardware Chassis`: X205 / VEGMAN; 

Мы получаем данные со стенда с шестнадцатью узлами (имеем замеры для nodes $= 4, \cdots\,16$ ) для различных комбинаций оставшихся признаков. Таким образом, математически, *замер* можно определить следующим образом $$(\,\boldsymbol{v}_{i_0,i_1,i_2,i_3,i_4,i_5}\;|\;nodes(i_0),\,size(i_1),\,type(i_2),\,protection(i_3),\,HW\_chassis(i_4),\,SSD(i_5)\,),\;\boldsymbol{v}_{i_0,i_1,i_2,i_3,i_4,i_5}\in R$$

На имеющихся данных необходимо обучить модель $M(nodes, size, type, protection, HW\_chassis, SSD)$, в первую очередь нас интересует экстраполяционная способность этой модели по параметру $nodes$: в данных $nodes \in [4,16]$, хотим знать поведение целевого значения для $nodes \in [4, 100]$ и всевозможных комбинаций остальных признаков.

## Реализация

### Конфигурирование

В словаре `doc` скрипта `Documentation` зафиксированы все возможные значения признаков. Ключи словаря разбиты на два массива - `cont_params`, `cat_params` - непрерывные и категориальные признаки соответственно. Словарь необходим для синхронизации и автоматизации обновления параметров и их значений по всему проекту, унификации системных наименований признаков.

Под скриптом расположен .md блок с обозначением общего вида файла замера, допустимого к использованию в считывающем скрипте `Reader`.

### Чтение замеров

Данные (замеры) поступают в виде .txt / .md файлов, в зависимости от политики защиты (`protection`) вид записи в файл может отличаться. 

Для чтения данных написан скрипт `Reader`, читающий все файлы папки "Correct sizes" и записывающий необходимые данные в словарь `Data`, в котором ключи - параметры замера. `Data` имеет следующую структуру: $$\left\{\begin{pmatrix}protection \\ size \\ nodes \\ w/r \\ HW\_chassis \\ SSD \end{pmatrix}:\;\begin{Bmatrix}MB/s: & list \\ avg: & list\\ min: & list \\ med: & list \\ max: & list \\ p(90): & list \\ p(95): & list \\ op/s: & list \\ op/s\_loss: & list \\ SUM\_MB/s: & float \\ SUM\_op/s: & float\end{Bmatrix}\right\}$$

*принцип работы:*

С помощью `glob.glob` собираем все .txt и .md файлы из папки, конкатенируем их.

Проходимся по файлам циклом

Обрабатываем файл так: создаём list, добавляем туда все строки из файла и с while проходимся по строчкам: мы знаем, что каждый замер начинается с '###' и имеет фиксированный тип, по ходу записываем признаки в ключ временного словаря `Params`, а целевое значение и побочную информацию (min, max, avg, et c.) во временный словарь `Input`. Так, после обработки замера нам нужно лишь присвоить `Data[tuple(Params.values())] = Input`.

### Выгрузка данных

Скрипты `Converter` и `Writer` конвертируют словарь `Data` в список и выгружают его в виде .xlsx файла

*принцип работы:*

`Converter` циклом переписывает значения из словаря в новый список. 
`Writer` переводит list `Data` полученный в результате работы конвертера в `pd.DataFrame` и методом `.to_excel` выгружает .xlsx док.

### Отрисовка данных

Как для сырых данных так и для предсказанных выводятся графики. Скрипт предоставляет возможность сравнить значения производительности для двух различных наборов данных. Переменная `parametr` определяет, какой признак будет по оси абсцисс - он принимает значения `size`, `nodes`. Словари `fix_params`, `fix_params_2` - фиксируют оставшиеся признаки.

### Препроцессинг данных

Перед обучением модели необходимо предобработать данные:    
Для категориальных признаков выбран метод `One Hot Encoder`.

#### One Hot Encoder

Для реализации написана функция `features_transformation(Data: dict) -> pd.DataFrame`. Encoding реализован `pd` методом `.get_dummies`, в связи с этим есть некоторые проблемы с требованием ко входным данным. Поскольку мы планируем использовать кодированные данные для обучения, то их размерность должна быть *чётко фиксирована*. Чтобы решить эту проблему написан скрипт `Documentation` он фиксирует в словаре `doc` вариации значений параметров и добавляет в `Data` записи, перебирающие всевозможные комбинации признаков. Функция `features_transformation` обрабатывает это и выдаёт DataFrame, соответствующий входному словарю. Это решает проблему зависимости размерности признаков при действии функции от входного словаря.


Из предметной области знаем, что: $$\text{target value}\;|\;\text{nodes}\sim \log (\text{nodes})$$ 
Из анализа поведения зависимости таргет значения от параметра `size` стало ясно, что: $$\text{target value}\;|\;\text{size}\sim \lambda\cdot\text{size} + N(\text{size};\;\mu = 1e2,\;\sigma^2 = 25e2)$$    
Где $N(x;\;\mu,\;\sigma^2)$ - плотность нормального распределения, $\lambda$ - вещественное число.  
Введём для `numerical` параметров - `nodes, size` - базисные функции:

$$\boldsymbol{\phi}_{size}(s) = \begin{pmatrix}
e^{-(s - 1e2)^2 / 25e2}\\
1\\
s
\end{pmatrix}\in R^{3};\;\boldsymbol{\phi}_{nodes}(n) = \begin{pmatrix}
\log(n)\\
1\\
n
\end{pmatrix}\in R^3$$

Для `categorial` параметров - `protection, type, HW_chassis` - One Hot Encoder


Таким образом, модель приняла вид:

$$\hat{v}\,(i_0\,i_1\,\cdots\,i_5) = \boldsymbol{w}^\top \cdot\begin{pmatrix}
\boldsymbol{\phi}_{nodes}(nodes\,(i_0))\\
\boldsymbol{\phi}_{size}(size\,(i_1))\\
\boldsymbol{\text{OHE}}(protection\,(i_2),\;type\,(i_3),\;HW\_chassis\,(i_4))\\
SSD\,(i_5)
\end{pmatrix}$$

где $\boldsymbol{\text{OHE}}$ - кодирование наших `cat.` параметров.